# Test new evolution class with time as free parameter

In [ ]:
import holodeck as holo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from holodeck import utils, cosmo
from holodeck import plot as hplot
import holodeck.accretion
from holodeck.constants import MSOL, PC, YR, MPC, GYR, MYR
import gwb as psgwb
import seaborn as sns

holo.log.setLevel(holo.log.INFO)

In [ ]:
ECCEN_INIT = 0.01
F_EDD = 0.1
NSTEPS = 100

pop = holo.population.Pop_Illustris(select=10)
size = pop.size
eccen = np.ones(size) * ECCEN_INIT
pop = holo.population.Pop_Illustris(eccen=eccen, select=10)

hards = [
    holo.hardening.Hard_GW,
    holo.hardening.CBD_Torques(allow_softening=True),
    holo.hardening.Sesana_Scattering(),
    holo.hardening.Dynamical_Friction_NFW(attenuate=True),
]

acc = holo.accretion.Accretion(
    accmod='Siwek22', f_edd=F_EDD, subpc=True, evol_mass=False, edd_lim=1.0,
)

evo_acc = holo.evolution.New_Evolution(pop, hards, debug=True, acc=acc)
evo_acc.evolve()

# Check evolution of mass, separation and eccentricity

In [ ]:
evo=evo_acc

fs = 30
lw = 6
figwidth = 15
figheight = 10
ticksize = 10
tickwidth = 4

fig,axes = plt.subplots(2,2, figsize=(figwidth,figheight))

axes = axes.flatten()
f_ind = 0
j = 0
for l_ind in evo._last_index:
    m1 = evo.mass[f_ind:l_ind].T[0]
    m2 = evo.mass[f_ind:l_ind].T[1]
    #print("m1/MSOL = ", m1/MSOL)
    t = evo.tlook[f_ind:l_ind]
    sepa = evo.sepa[f_ind:l_ind]
    mtot = (m1+m2)/MSOL
    eccen = evo.eccen[f_ind:l_ind]
    ind_subpc = evo.sepa[f_ind:l_ind] <= 1*PC
    #calculate the time that has passed since binary formed
    
    
    if len(mtot[ind_subpc]) > 1:
        color = sns.color_palette('colorblind')[j]
        t_formation = t[ind_subpc][0] - t[ind_subpc]
        axes[0].loglog(t_formation/MYR,mtot[ind_subpc], color=color, alpha = 0.7, linewidth=lw)
        #map out the predicted mass growth
        m0 = mtot[ind_subpc][0]
        m_evol = [m0*MSOL]
        for dt in np.diff(t_formation):
            mdot = acc.mdot_eddington(m_evol[-1], eps=0.1)
            dm = mdot * dt
            m_evol.append(m_evol[-1] + dm)
        m_evol = np.array(m_evol)/MSOL
        #map out the predicted mass growth
        axes[0].loglog(t_formation/MYR,m_evol, '--', color='magenta', alpha = 0.7, linewidth=0.5*lw)
    
        axes[1].loglog(sepa[ind_subpc]/PC,mtot[ind_subpc], color=color, alpha = 0.7, linewidth=lw)
        axes[2].loglog(t_formation/MYR, sepa[ind_subpc]/PC, color=color, alpha = 0.7, linewidth=lw)
        axes[3].loglog(sepa[ind_subpc]/PC, eccen[ind_subpc], color=color, alpha = 0.7, linewidth=lw)
        j+=1
    f_ind = l_ind+1

axes[0].invert_xaxis()
for ax in axes:
    ax.invert_xaxis()
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)
    ax.grid(which='both', color='k', linestyle='-', linewidth=0.5)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

axes[0].set_ylabel(r'$M_{\rm b}/M_{\odot}$', fontsize=1.2*fs)
axes[0].set_xlabel(r'$t_{\rm formation}/\rm{Myr}$', fontsize=1.2*fs)
axes[1].set_xlabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
axes[1].set_ylabel(r'$M_{\rm b}/M_{\odot}$', fontsize=1.2*fs)
axes[1].axvline(x = 1, linestyle = '--', alpha = 0.5, linewidth=lw, color='gray')
axes[2].set_xlabel(r'$t_{\rm look}/\rm{Myr}$', fontsize=1.2*fs)
axes[2].set_ylabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
axes[3].set_xlabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
axes[3].set_ylabel(r'$e_{\rm b}$', fontsize=1.2*fs)

plt.tight_layout()


# Hardening Rates

In [ ]:
evo=evo_acc

hard_cbd = holo.hardening.CBD_Torques(allow_softening=True)

import seaborn as sns
_AGE_UNIVERSE_GYR = cosmo.age(0.0).to('s').value

fs = 40
lw = 6
figwidth = 10
figheight = 8
ticksize = 10
tickwidth = 4

fig,axes = plt.subplots(3,2, figsize=(2*figwidth,3*figheight))

axes = axes.flatten()

f_ind = 0
j = 0
for k,l_ind in enumerate(evo._last_index):
    m1 = evo.mass[f_ind:l_ind].T[0]
    m2 = evo.mass[f_ind:l_ind].T[1]
    qb = m2/m1
    if np.any(qb>1):
        qb[qb>1] = 1./qb[qb>1]
    t = evo.tlook[f_ind:l_ind]
    sepa = evo.sepa[f_ind:l_ind]
    mtot = (m1+m2)/MSOL
    eccen = evo.eccen[f_ind:l_ind]
    mdot = np.sum(evo.mdot[f_ind:l_ind],axis=-1)
    redz = cosmo.tage_to_z(_AGE_UNIVERSE_GYR - t)
    #print("redz = ", redz)
    frst_orb = utils.kepler_freq_from_sepa(mtot*MSOL, sepa)
    fobs = utils.fobs_from_frst(frst_orb, redz)
    #analytical gw hardening rate
    dadt_gw = -1*utils.gw_hardening_rate_dadt(m1, m2, sepa, eccen=eccen)
    #CBD hardening rate
    dadt_cbd = -1*holo.hardening._Siwek2023.dadt(qb, eccen) * sepa * (mdot/(mtot*MSOL))
    #CBD hardening rate from evo
    dadt_evo_cbd = -1*evo.dadt[f_ind:l_ind][:,1]
    #dadt from evo, including all other hardening rates
    dadt_combined_evo = -1*np.sum(evo.dadt[f_ind:l_ind],axis=-1)

    ind_subpc = evo.sepa[f_ind:l_ind] <= 1*PC

    if len(mtot[ind_subpc]) > 1:
        #calculate the time that has passed since binary hit 1 parsec separation
        t_formation = t[ind_subpc][0] - t[ind_subpc]
        #print("np.shape(evo.dadt[f_ind:l_ind]) = ", np.shape(evo.dadt[f_ind:l_ind]))
        #only plot sub-parsec binaries
        color = sns.color_palette('colorblind')[j]
        if j == 0:
            axes[0].loglog(fobs[ind_subpc], dadt_gw[ind_subpc], '--', label = r'$\dot{a}_{\rm gw}$', color='magenta', alpha = 0.7, linewidth=0.5*lw)
            axes[0].loglog(fobs[ind_subpc], dadt_combined_evo[ind_subpc], label = 'binary%d' %k, color=color, alpha = 0.7, linewidth=lw)
        else:
            axes[0].loglog(fobs[ind_subpc], dadt_combined_evo[ind_subpc], label = 'binary%d' %k, color=color, alpha = 0.7, linewidth=lw)
            axes[0].loglog(fobs[ind_subpc], dadt_gw[ind_subpc], '--', color='magenta', alpha = 0.7, linewidth=0.5*lw)
        
        axes[1].loglog(t_formation/YR, fobs[ind_subpc], color=color, alpha=0.7, linewidth=lw)
        
        if j == 0:
            axes[2].loglog(fobs[ind_subpc], dadt_cbd[ind_subpc], '--', label = r'$\dot{a}_{\rm cbd}$', color='magenta', alpha = 0.7, linewidth=0.5*lw)
            axes[2].loglog(fobs[ind_subpc], dadt_evo_cbd[ind_subpc], label = 'binary%d' %k, color=color, alpha = 0.7, linewidth=lw)
        else:
            axes[2].loglog(fobs[ind_subpc], dadt_evo_cbd[ind_subpc], label = 'binary%d' %k, color=color, alpha = 0.7, linewidth=lw)
            axes[2].loglog(fobs[ind_subpc], dadt_cbd[ind_subpc], '--', color='magenta', alpha = 0.7, linewidth=0.5*lw)
        
        if j == 0:
            axes[3].loglog(fobs[ind_subpc], -1*dadt_cbd[ind_subpc], '--', label = r'$\dot{a}_{\rm cbd}$', color='magenta', alpha = 0.7, linewidth=0.5*lw)
            axes[3].loglog(fobs[ind_subpc], -1*dadt_evo_cbd[ind_subpc], label = 'binary%d' %k, color=color, alpha = 0.7, linewidth=lw)
        else:
            axes[3].loglog(fobs[ind_subpc], -1*dadt_evo_cbd[ind_subpc], label = 'binary%d' %k, color=color, alpha = 0.7, linewidth=lw)
            axes[3].loglog(fobs[ind_subpc], -1*dadt_cbd[ind_subpc], '--', color='magenta', alpha = 0.7, linewidth=0.5*lw)
        
        axes[4].loglog(fobs[ind_subpc], qb[ind_subpc], label = 'binary%d' %k, color=color, alpha = 0.7, linewidth=lw)
        
        axes[5].loglog(fobs[ind_subpc], eccen[ind_subpc], label = 'binary%d' %k, color=color, alpha = 0.7, linewidth=lw)


        
        j += 1


    f_ind = l_ind+1

axes[0].legend(fontsize=0.7*fs)

#axes[0].invert_xaxis()
for ax in axes:
    #ax.invert_xaxis()
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)
    ax.grid(which='both', color='k', linestyle='-', linewidth=0.5)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

axes[0].set_ylabel(r'$-\dot{a}_{\rm b}\, [\rm{cm/s}]$', fontsize=1.2*fs)
axes[0].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)
axes[1].set_xlabel(r'$t_{\rm formation}/\rm{Myr}$', fontsize=1.2*fs)
axes[1].set_ylabel(r'$f_{\rm b}/\rm{Hz}$', fontsize=1.2*fs)

axes[2].set_ylabel(r'$-\dot{a}_{\rm b}\, [\rm{cm/s}]$', fontsize=1.2*fs)
axes[2].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)

axes[3].set_ylabel(r'$+\dot{a}_{\rm b}\, [\rm{cm/s}]$', fontsize=1.2*fs)
axes[3].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)

axes[4].set_ylabel(r'$q_{\rm b}$', fontsize=1.2*fs)
axes[4].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)

axes[5].set_ylabel(r'$e_{\rm b}$', fontsize=1.2*fs)
axes[5].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)


# axes[1].axvline(x = 1, linestyle = '--', alpha = 0.5, linewidth=lw, color='gray')
# axes[1].set_xlim([1.e7,1.e-5])
# axes[2].set_xlabel(r'$t_{\rm look}/\rm{Myr}$', fontsize=1.2*fs)
# axes[2].set_ylabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
# axes[3].set_xlabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
# axes[3].set_ylabel(r'$e_{\rm b}$', fontsize=1.2*fs)

plt.tight_layout()


# Check that evolution is consistent WITHOUT accretion and torques

In [ ]:
import holodeck as holo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from holodeck import utils, cosmo
from holodeck import plot as hplot
import holodeck.accretion
from holodeck.constants import MSOL, PC, YR, MPC, GYR, MYR
import gwb as psgwb
# import seaborn as sns

holo.log.setLevel(holo.log.INFO)

ECCEN_INIT = 0.01
NSTEPS = 100

pop = holo.population.Pop_Illustris(select=10)
size = pop.size
eccen = np.ones(size) * ECCEN_INIT
pop = holo.population.Pop_Illustris(eccen=eccen, select=10)

# redz = cosmo.a_to_z(pop.scafa)

hards = [
    holo.hardening.Hard_GW,
    holo.hardening.Sesana_Scattering(),
    holo.hardening.Dynamical_Friction_NFW(attenuate=True),
]

evo_noacc = holo.evolution.New_Evolution(pop, hards, debug=True, acc=None)
evo_noacc.evolve()

In [ ]:
evo=evo_noacc

fs = 30
lw = 4
figwidth = 15
figheight = 10
ticksize = 10
tickwidth = 4

fig,axes = plt.subplots(2,2, figsize=(figwidth,figheight))

axes = axes.flatten()
f_ind = 0
j = 0
for l_ind in evo._last_index:
    m1 = evo.mass[f_ind:l_ind].T[0]
    m2 = evo.mass[f_ind:l_ind].T[1]
    #print("m1/MSOL = ", m1/MSOL)
    t = evo.tlook[f_ind:l_ind]
    sepa = evo.sepa[f_ind:l_ind]
    mtot = (m1+m2)/MSOL
    eccen = evo.eccen[f_ind:l_ind]
    color = sns.color_palette('colorblind')[j]
    axes[0].loglog(t/MYR,mtot, color=color, alpha = 0.7, linewidth=lw)
    axes[1].loglog(sepa/PC,mtot, color=color, alpha = 0.7, linewidth=lw)
    axes[2].loglog(t/MYR, sepa/PC, color=color, alpha = 0.7, linewidth=lw)
    axes[3].loglog(sepa/PC, eccen, color=color, alpha = 0.7, linewidth=lw)
    # print("sepa = [%.4e, %.4e]" %(sepa[0]/PC, sepa[-1]/PC))
    # print("mtot = [%.4e, %.4e]" %(mtot[0], mtot[-1]))
    j+=1
    f_ind = l_ind+1

for ax in axes:
    ax.invert_xaxis()
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)
    ax.grid(which='both', color='k', linestyle='-', linewidth=0.5)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
#ax.legend()

axes[0].set_ylabel(r'$M_{\rm b}/M_{\odot}$', fontsize=1.2*fs)
axes[0].set_xlabel(r'$t_{\rm look}/\rm{Myr}$', fontsize=1.2*fs)
axes[1].set_xlabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
axes[1].set_ylabel(r'$M_{\rm b}/M_{\odot}$', fontsize=1.2*fs)
axes[1].axvline(x = 1, linestyle = '--', alpha = 0.5, linewidth=lw, color='gray')
axes[1].set_xlim([1.e7,1.e-5])
axes[2].set_xlabel(r'$t_{\rm look}/\rm{Myr}$', fontsize=1.2*fs)
axes[2].set_ylabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
axes[3].set_xlabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
axes[3].set_ylabel(r'$e_{\rm b}$', fontsize=1.2*fs)

plt.tight_layout()


# Compare old evolution to new evolution class

In [ ]:
import holodeck as holo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from holodeck import utils, cosmo
from holodeck import plot as hplot
import holodeck.accretion
from holodeck.constants import MSOL, PC, YR, MPC, GYR, MYR
import gwb as psgwb
# import seaborn as sns

holo.log.setLevel(holo.log.INFO)

ECCEN_INIT = 0.01
F_EDD = 0.1
NSTEPS = 100

pop = holo.population.Pop_Illustris(select=10)
size = pop.size
eccen = np.ones(size) * ECCEN_INIT
pop = holo.population.Pop_Illustris(eccen=eccen, select=10)

# redz = cosmo.a_to_z(pop.scafa)

hards_old = [
    holo.hardening.Hard_GW,
    holo.hardening.CBD_Torques(allow_softening=False),
    holo.hardening.Sesana_Scattering(),
    holo.hardening.Dynamical_Friction_NFW(attenuate=True),
]

hards = [
    holo.hardening.Hard_GW,
    holo.hardening.CBD_Torques(allow_softening=True),
    holo.hardening.Sesana_Scattering(),
    holo.hardening.Dynamical_Friction_NFW(attenuate=True),
]

acc = holo.accretion.Accretion(
    accmod='Siwek22', f_edd=F_EDD, subpc=True, evol_mass=True, edd_lim=1.0,
)

evo_old = holo.evolution.Evolution(pop, hards_old, nsteps = NSTEPS, debug=True, acc=acc)
evo_old.evolve()

evo_new = holo.evolution.New_Evolution(pop, hards, debug=True, acc=acc)
evo_new.evolve()


In [ ]:
fs = 30
lw = 6
figwidth = 15
figheight = 10
ticksize = 10
tickwidth = 4

fig,axes = plt.subplots(2,2, figsize=(figwidth,figheight))

axes = axes.flatten()

f_ind = 0
k_old = 0
k_new = 0
nb = 3
ab_lim = 1*PC
for j,l_ind in zip([nb],[evo_new._last_index[nb]]):#enumerate(evo_new._last_index):
    for label,evo,color,ls in zip(['old','new'],[evo_old,evo_new],\
                            ['crimson','seagreen'],['-','-']):
        print("1. label = ", label)
        if label == 'new':
            ind_subpc = evo.sepa[f_ind:l_ind] <= ab_lim
            m1 = evo.mass[f_ind:l_ind].T[0][ind_subpc]
            m2 = evo.mass[f_ind:l_ind].T[1][ind_subpc]
            #print("m1/MSOL = ", m1/MSOL)
            t = evo.tlook[f_ind:l_ind][ind_subpc]
            sepa = evo.sepa[f_ind:l_ind][ind_subpc]
            mtot = (m1+m2)
            eccen = evo.eccen[f_ind:l_ind][ind_subpc]
        if label == 'old':
            ind_subpc = evo.sepa[j,:] <= ab_lim
            m1 = evo.mass[j,:].T[0][ind_subpc]
            m2 = evo.mass[j,:].T[1][ind_subpc]
            t = evo.tlook[j,:][ind_subpc]
            sepa = evo.sepa[j,:][ind_subpc]
            mtot = (m1+m2)
            eccen = evo.eccen[j,:][ind_subpc]
        
        if len(mtot) > 1:
            print("2. label = ", label)
            if label == 'new':
                k_new += 1
            if label == 'old':
                k_old += 1
            if (k_old <= 1 and label == 'old') or (k_new <= 1 and label == 'new'):
                t_formation = t[0] - t
                #map out the predicted mass growth
                m0 = mtot[0]
                m_evol = [m0]
                for dt in np.diff(t_formation):
                    mdot = acc.mdot_eddington(m_evol[-1], eps=0.1)
                    dm = mdot * dt
                    m_evol.append(m_evol[-1] + dm)
                m_evol = np.array(m_evol)
                #map out the predicted mass growth

                axes[0].loglog(t_formation/MYR, mtot/MSOL, ls, label=label, \
                    color=color, alpha = 0.7, linewidth=lw)
                axes[0].loglog(t_formation/MYR,m_evol/MSOL, '--', \
                    color='magenta', alpha = 0.7, linewidth=0.5*lw)
            
                axes[1].loglog(sepa/PC, mtot/MSOL, ls, label=label, \
                    color=color, alpha = 0.7, linewidth=lw)
                axes[2].loglog(t_formation/MYR, sepa/PC, ls, label=label, \
                    color=color, alpha = 0.7, linewidth=lw)
                axes[3].loglog(sepa/PC, eccen, ls, label=label, \
                    color=color, alpha = 0.7, linewidth=lw)

                if label == 'new':
                    f_ind = l_ind+1


axes[0].invert_xaxis()
axes[2].invert_xaxis()
for ax in axes:
    ax.invert_xaxis()
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)
    ax.grid(which='both', color='k', linestyle='-', linewidth=0.5)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

axes[0].set_ylabel(r'$M_{\rm b}/M_{\odot}$', fontsize=1.2*fs)
axes[0].set_xlabel(r'$t_{\rm formation}/\rm{Myr}$', fontsize=1.2*fs)
axes[1].set_xlabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
axes[1].set_ylabel(r'$M_{\rm b}/M_{\odot}$', fontsize=1.2*fs)
axes[1].axvline(x = 1, linestyle = '--', alpha = 0.5, linewidth=lw, color='gray')
axes[2].set_xlabel(r'$t_{\rm look}/\rm{Myr}$', fontsize=1.2*fs)
axes[2].set_ylabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
axes[3].set_xlabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
axes[3].set_ylabel(r'$e_{\rm b}$', fontsize=1.2*fs)

axes[3].legend(fontsize=fs)

plt.tight_layout()


    

In [ ]:
hard_cbd = holo.hardening.CBD_Torques(allow_softening=True)

import seaborn as sns
_AGE_UNIVERSE_GYR = cosmo.age(0.0).to('s').value

fs = 20
lw = 3
figwidth = 7
figheight = 5
ticksize = 10
tickwidth = 4

fig,axes = plt.subplots(4,1, figsize=(figwidth,4*figheight))

axes = axes.flatten()

f_ind = 0
k_old = 0
k_new = 0
nb = 3
ab_lim = 1*PC
for j,l_ind in zip([nb],[evo_new._last_index[nb]]):#enumerate(evo_new._last_index):
    for label,evo,color,ls in zip(['old','new'],[evo_old,evo_new],\
                            ['crimson','seagreen'],['-','-']):
        print("1. label = ", label)
        if label == 'new':
            ind_subpc = evo.sepa[f_ind:l_ind] <= ab_lim
            m1 = evo.mass[f_ind:l_ind].T[0][ind_subpc]
            m2 = evo.mass[f_ind:l_ind].T[1][ind_subpc]
            #print("m1/MSOL = ", m1/MSOL)
            t = evo.tlook[f_ind:l_ind][ind_subpc]
            sepa = evo.sepa[f_ind:l_ind][ind_subpc]
            mtot = (m1+m2)
            eccen = evo.eccen[f_ind:l_ind][ind_subpc]
            mdot = np.sum(evo.mdot[f_ind:l_ind][ind_subpc],axis=-1)
            # #CBD hardening rate from evo
            dadt_evo_cbd = -1*evo.dadt[f_ind:l_ind][ind_subpc][:,1]
            #dadt from evo, including all other hardening rates
            dadt_combined_evo = -1*np.sum(evo.dadt[f_ind:l_ind][ind_subpc],axis=-1)
        if label == 'old':
            ind_subpc = evo.sepa[j,:] <= ab_lim
            m1 = evo.mass[j,:].T[0][ind_subpc]
            m2 = evo.mass[j,:].T[1][ind_subpc]
            t = evo.tlook[j,:][ind_subpc]
            sepa = evo.sepa[j,:][ind_subpc]
            mtot = (m1+m2)
            eccen = evo.eccen[j,:][ind_subpc]
            mdot = np.sum(evo.mdot[j,:][ind_subpc],axis=-1)
            dadt_combined_evo = -1*evo.dadt[j,:][ind_subpc]
            # #CBD hardening rate from evo
            dadt_evo_cbd = -1*evo._dadt_1[j,:][ind_subpc]
    
        qb = m2/m1
        if np.any(qb>1):
            qb[qb>1] = 1./qb[qb>1]
        redz = cosmo.tage_to_z(_AGE_UNIVERSE_GYR - t)
        frst_orb = utils.kepler_freq_from_sepa(mtot, sepa)
        fobs = utils.fobs_from_frst(frst_orb, redz)
        #analytical gw hardening rate
        dadt_gw = -1*utils.gw_hardening_rate_dadt(m1, m2, sepa, eccen=eccen)
        #CBD hardening rate
        dadt_cbd = -1*holo.hardening._Siwek2023.dadt(qb, eccen) * sepa * (mdot/(mtot))

        if len(mtot) > 1:
            print("2. label = ", label)
            #calculate the time that has passed since binary hit 1 parsec separation
            t_formation = t[0] - t

            if label == 'new':
                ax0_twin = axes[0].twiny()
                ax0_twin.loglog(sepa/PC, dadt_combined_evo, color=color, alpha = 0.0, linewidth=lw)

            axes[0].loglog(fobs, dadt_combined_evo, label = label, color=color, alpha = 0.7, linewidth=lw)
            axes[0].loglog(fobs, dadt_gw, '--', color='magenta', alpha = 0.7, linewidth=0.5*lw)

            axes[1].loglog(fobs, dadt_evo_cbd, label = label, color=color, alpha = 0.7, linewidth=lw)
            axes[1].loglog(fobs, dadt_cbd, '--', color='magenta', alpha = 0.7, linewidth=0.5*lw)

            axes[2].loglog(fobs, qb, label = label, color=color, alpha = 0.7, linewidth=lw)
            
            axes[3].loglog(fobs, eccen, label = label, color=color, alpha = 0.7, linewidth=lw)



    f_ind = l_ind+1

axes[0].legend(fontsize=fs)

#axes[0].invert_xaxis()
for ax in axes:
    #ax.invert_xaxis()
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)
    ax.grid(which='both', color='k', linestyle='-', linewidth=0.5)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

for ax in [ax0_twin]:
    ax.invert_xaxis()
    ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
    ax.tick_params(axis='both', which='minor', direction='inout', size=0.7*ticksize, width=0.7*tickwidth)
    ax.grid(which='both', color='k', linestyle='-', linewidth=0.5)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

ax0_twin.set_xlabel(r'$a_{\rm b}\, \rm{[PC]}$', fontsize=1.2*fs)
axes[0].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)
axes[0].set_ylabel(r'$-\dot{a}_{\rm b}\, [\rm{cm/s}]$', fontsize=1.2*fs)

axes[1].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)
axes[1].set_ylabel(r'$-\dot{a}_{\rm b,cbd}\, [\rm{cm/s}]$', fontsize=1.2*fs)

axes[2].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)
axes[2].set_ylabel(r'$q_{\rm b}$', fontsize=1.2*fs)

axes[3].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)
axes[3].set_ylabel(r'$e_{\rm b}$', fontsize=1.2*fs)


# axes[3].set_ylabel(r'$+\dot{a}_{\rm b}\, [\rm{cm/s}]$', fontsize=1.2*fs)
# axes[3].set_xlabel(r'$f_{\rm orb,obs}/\rm{Hz}$', fontsize=1.2*fs)

# axes[1].axvline(x = 1, linestyle = '--', alpha = 0.5, linewidth=lw, color='gray')
# axes[1].set_xlim([1.e7,1.e-5])
# axes[2].set_xlabel(r'$t_{\rm look}/\rm{Myr}$', fontsize=1.2*fs)
# axes[2].set_ylabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
# axes[3].set_xlabel(r'$a_{\rm b}/\rm{PC}$', fontsize=1.2*fs)
# axes[3].set_ylabel(r'$e_{\rm b}$', fontsize=1.2*fs)

plt.tight_layout()
plt.savefig('test_new_old_evo.png', facecolor='w', bbox_inches='tight')


In [ ]:
import holodeck as holo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from holodeck import utils, cosmo
from holodeck import plot as hplot
import holodeck.accretion
from holodeck.constants import MSOL, PC, YR, MPC, GYR, MYR
import gwb as psgwb
# import seaborn as sns

holo.log.setLevel(holo.log.INFO)

ECCEN_INIT = 0.01
F_EDD = 0.1
NSTEPS = 100

pop = holo.population.Pop_Illustris(select=10)
size = pop.size
eccen = np.ones(size) * ECCEN_INIT
pop = holo.population.Pop_Illustris(eccen=eccen, select=10)

# redz = cosmo.a_to_z(pop.scafa)

hards_old = [
    holo.hardening.Hard_GW,
    holo.hardening.CBD_Torques(allow_softening=False),
    holo.hardening.Sesana_Scattering(),
    holo.hardening.Dynamical_Friction_NFW(attenuate=True),
]

acc = holo.accretion.Accretion(
    accmod='Siwek22', f_edd=F_EDD, subpc=True, evol_mass=True, edd_lim=1.0,
)

evo_old = holo.evolution.Evolution(pop, hards_old, nsteps = NSTEPS, debug=True, acc=acc)
evo_old.evolve()

